In [ ]:
#This notebook contains functions necessary to implement Shamir's secret sharing
#Note that this code and the code that will be implemented into the project uses a linear representation instead of a finite field. 
#For any enterprise development the code must be changed to include a finite field build around a significantly large prime such as 2^127 -1 

In [21]:
#This cell contains necessary imports
import numpy as np
from scipy.interpolate import lagrange 
import pandas as pd
from random import seed
from random import randint
from numpy.polynomial.polynomial import Polynomial
import Crypto
#bytes object initialization 
from Crypto.PublicKey import RSA
#import bcrypt
import rsa
import random
import math

In [64]:
def main():
    S_list = create_secret_value_table()
    #generates a list of secret indicies
    RSA_list = generate_RSA_keypairs()
    #generates a dataframe of public private keypairs
    S_df = pd.DataFrame({"Secret ID": S_list})
    S_RSA_DF = pd.concat([S_df, RSA_list], axis = 1)
    S_RSA_DF.to_csv('RSA_Keys.csv')
    
    
    #concatonates the two frames creating a pair between secret ID and each set of keys
        
main()

In [3]:
def block_processing(block):
    agent_list = we_need_agents()
    (S, fC1, fC2) = generate_function_coefficients()
    share_list = generate_Shamir_Shares(agent_list, S, fC1, fC2)
    index = Block_DF.loc[Block_DF['Block_Number'] == block].index[0]
    block_hash = Block_DF.loc[BLock_DF['Block_Number'] == block]['Raw_Bytes'][index]
    encrypted_block = encrypt_the_block(block, S)
    return share_list, encrypted_block

In [43]:
S = 792
block = 104
index = Block_DF.loc[Block_DF['Block_Number'] == block].index[0]
block_hex = Block_DF.loc[Block_DF['Block_Number'] == block]['Raw_bytes'][index]
unencrypted_hex_list = break_the_block(block_hex)
encrypted_parts = []
for i in unencrypted_hex_list:
    encrypted_pt = encrypt_the_block(i, S)
    encrypted_parts.append(encrypted_pt)
encrypted_parts

[b'\x12 \x8d\xd0\x81f\xf9SR\xca\xe7\xaeIz\xcct?\xe4\xc9r\xf1\xd5\xf6\xe0\xf9\xeb\xfb\xfe\x9e\xca=\x95\xe3\x8c\xc8\x0f\x80\xc1\xa6"\xde!\xdfw\xa1\xd9\xb6\x08\xb0\x15G\x14\x00,\x1cN\xc6\xfeT&M\xea\xe1\xf5\x8eZ\x1e$\xcb\x03[\x84\x9c\xd3\xc1\xc6\xd3\xf7\x17\xb4\x87\x06\xc6\xef\xd1LoO\x9d\x9a+&\xc6\xecG\x8a\xae0L\xa9\xd5\xd0iP\xe7\x90\x80\xa9#\xfd\x981\xe8\xd2\xa2\x8d\xf1\xd7E"3\x81u\xf6t\x90\xcb-h\x89\xc9\x90\xae\xe3\xf3\xe0\x88+\xbc\x8d\xda\x01\xf48j\xe2\x03\x9bY\xea\x13j\xe4\\|A\xf3R\xdb\xfe\xf8\x01f\xc5\xb8b\xd2\x9b\x98\xad`\x84\xf2|(z=&wBrI\xb6\x84\xc3\xa1t\xf5(:\xb8X9\xfeMB\x08\x9a\x83Z\xc8\xe1\xear\x05E\xcb\xe8\xbdX,\xb0\xf3\xbd2\xf8n`\xd7\xb4\xf3\x92.z\x1dU9o\x05\xf5i`iE3y\x9e\xf2l\x8a\xe1\xb93-\xc4\xba\xfb\xb6 \x83\xa0mB\xc9\x1f\x8bL\x8ay\x8br\xea\xd4\xb6\xe1\xffw\xc6!\xb9\x8f\x0f\xa2\x84\xd9\'\xafuY3"\xea.\xbc\xc5xA\xdf\xeb-\xd6\x95\xab\xd0\x0f\xf8/tc\x18$=T\xef\xb9\xad\tI\tx%\xba\x0f\xd3\x08\xd9[\x88\xcd\x97\xbf\x9e\x93\xe3A9\xd1q)G\x8d\x12\x9f:\x94\xf2qb\\y\n|\xd4]r;\x08\xd5wH\x

In [31]:
#This cell is used to test the encryption and block processing. 
(share_list, encrypted_block) = block_processing(block_list[1])
#(share_list, encrypted_block) = block_processing('00000020ec80c6c8f61a7810eed8d1b08b0870f34a49d59d336e4a069b7cf6f6a50ef335166970c468a1ff44df44e7b0c2d20c5054cf430e0516b785f60993683b52ba893e6b235effff7f200000000002020000000001010000000000000000000000000000000000000000000000000000000000000000ffffffff0401660101ffffffff02d0fe052a01000000160014e6e0768d5239cc1446fbf32cd825adf05b5110560000000000000000266a24aa21a9ed32209fa7aa328b352828204d4b41d1a90177efdf4c3bce51b9ccbe0a78647de90120000000000000000000000000000000000000000000000000000000000000000000000000020000000001013ccb4a45689fe7df424c98f3decf9eaebdd73c753d41894853d798c6a76ffeff0000000017160014b1a7d3b0d1011a3475c32949d2786676f0a376b3feffffff02300410240100000016001458db024dd71efaf0184926a4c85be3023fa0080400e1f50500000000160014e6e0768d5239cc1446fbf32cd825adf05b5110560247304402200e72240746ec045668b8a2e30ecd36a761ecc9bab1b2cc039dbb7ba12c7218820220019d9e63738e1b6932b6bf331c3514dd85ae4c2221f19ef458184907fa7b3455012102098cfd632f00010e78b712508cae2fc48c47a033b9a3c14e30b0eb3a074a5e4800000000')
#stringbs = '00000020ec80c6c8f61a7810eed8d1b08b0870f34a49d59d336e4a069b7cf6f6a50ef335166970c468a1ff44df44e7b0c2d20c5054cf430e0516b785f60993683b52ba893e6b235effff7f200000000002020000000001010000000000000000000000000000000000000000000000000000000000000000ffffffff0401660101ffffffff02d0fe052a01000000160014e6e0768d5239cc1446fbf32cd825adf05b5110560000000000000000266a24aa21a9ed32209fa7aa328b352828204d4b41d1a90177efdf4c3bce51b9ccbe0a78647de90120000000000000000000000000000000000000000000000000000000000000000000000000020000000001013ccb4a45689fe7df424c98f3decf9eaebdd73c753d41894853d798c6a76ffeff0000000017160014b1a7d3b0d1011a3475c32949d2786676f0a376b3feffffff02300410240100000016001458db024dd71efaf0184926a4c85be3023fa0080400e1f50500000000160014e6e0768d5239cc1446fbf32cd825adf05b5110560247304402200e72240746ec045668b8a2e30ecd36a761ecc9bab1b2cc039dbb7ba12c7218820220019d9e63738e1b6932b6bf331c3514dd85ae4c2221f19ef458184907fa7b3455012102098cfd632f00010e78b712508cae2fc48c47a033b9a3c14e30b0eb3a074a5e4800000000'
print(share_list)
print(encrypted_block)

[{2: 2764}, {1: 1485}, {8: 24046}, {3: 4691}, {7: 18879}]
b'\x18\xbe\x92]A\xec\x8d\xfb\x9a\x15N{\xcd\xfdWG\x14NX\xe4LK\xd7+z\xbf\xed\x8b\x0c1\xcf\xce\xcb\x00B7n10\xc7\xcfo\xf0\x99\xc1\x05\x03\xda\x9e\x0bu\x8ad\'\xddr\xe2\x9d\x1bH\xfa\x1a\xdd=\x97\x81\xebS\tg{\xb0,\'\x06\xf5cb\x14/\xa4\xcb\x1cA3\x9e*\x9a\x15\x02\x04\x89(9h#\xbe\x04\x888\xc9\x8fr\xc3\xa1\x02;#8k\xe3\x1e\xccNi\x8b"5\x1c\xfc\xf6\xbe\x1a\xf4\xb9\x8bfq\xb7\x01\x11\x96\xfd\xcc]]\x7fr\x90\xb1{\xd3k\x16I\x13?\xb3\xd78\xec\x1b\xb9e\x95\xf7\xc75\xca\x81\x89\xde\x9a\xa1&Z\xfa\x87BI\x97\xc9\xe6f\x11\n\x8f\xa2\x05\xc3nF+ \xd3>\x91&\x1d\x0e\xdc4\x87x\x16<\xeb\xcb\x05\x12\x0b\xf6|\xd9Z\xc4f~}+\xd0\x9d\x93\x84s\x1e_\xe3\xcb\xa9_\x1e>o\x0ey\xbb\xed\x82\xc8;\xf4$%c\xb7v\xa4\xc4g;\r\x0e\xdd\x93\xd8\x87\xf1|\x15@\xe4\xd2\xc2\xa1\xb0\xef\xb6\x87\xa4\xf5J5\xc1t7\x17\xc2\tg\xf8(\xbc\x95\xa5\xe9I\x1aN\x95\x8a\xa2\xed\x910\xd2\x8f<i\t\xf7\x0e\xe7\x9c\x87j\xbf\xa8n\x0eX\xdd\x9fB\xf7\xb0;8\x8e\xde\xfey\x91|\xe9\x93\\f\xc3e-/\x9a\xb6S\xe2v\xf2U$[<

In [24]:
#This method split the hex of the bitcoin blocks and breaks them into strings and returns them as a list. Due to RSA4096's length limitations we are capped at 501 bytes, thus we must trim the blocks and receive them as a list. 
def break_the_block(block_text):
    length = len(block_text)
    block_list = []
    for i in range(math.ceil(length/500)):
        substring = block_text[(500*i):(500*(i+1))]
        block_list.append(substring)
    return block_list

In [32]:
block = '00000020f67472a866d2b526a22ab132c2ad46578d58c968a3ff4542bc31b89778b2f237032f0b5b9a90f0bf15bf4396f9d8733b55a8cef1e8dbcdbced04e2a3ddae2b33cd11295effff7f200100000009020000000001010000000000000000000000000000000000000000000000000000000000000000ffffffff0401680101ffffffff02c059062a0100000017a9148b7a1532abef9a6cfc79cbb89a0fb171c73cb5d7870000000000000000266a24aa21a9edcc9e9dd007027c2ee8606ab265410f0af99d08038ac1a2298577f1fd47769c790120000000000000000000000000000000000000000000000000000000000000000000000000020000000001010de9e45636dc1b3178cdf43311bb3dc31d88e14e400450648a663f2dc9e64088000000001716001446d96422bc6661867305deb392a8c5c07d18f4c4feffffff02080410240100000017a914fb5ca92fef360be3009d9a8348212d35c91bf65c8700e1f5050000000017a914502f9eb830f1ac210a79147721b8c04f881dbe71870247304402204c8c27a2d230bb5b87c4732c4f0c3a63cfe61a5e3cdcc27e4ba3fa58400b314e02206e72f5670fa0d698ef1306b791794ef4a5188dd6b0555b6a10dc25b759fbdf7901210314cec2b73d7865554856c0c2e429be5566d8565e1f5ad9c6412b3022cbe65d1a6700000002000000000101eccdcf3ade33f3a3cb08c052f113c40fab83ee0fc53174112d908bef6801c97200000000171600142fbf079e4720979ceda468fe952d234f44b9b5a6feffffff0240787d010000000017a914f7b47eceb061184194882bd226c3495c5d1fbd8487d07e92220100000017a91461c49f7032686aa30366fa726ffba51e76eb62f787024730440220239bb3edab14720a7999c726e87d7a20567ea8e8c619e013ec128689bd3bbf51022011634f861bccc0ae85aaa9d02173e8539d91669bcd23e23674ce19dc0123bb19012102e8679bf5d9d4a4212fa991a08c99138dadf9ea58341224f5ad824e9f0622c9416700000002000000000101421651589f2eee1066cbef91ccb40c8c974b6ca8b199513554ae9791dcc00f07000000001716001446d96422bc6661867305deb392a8c5c07d18f4c4feffffff02080410240100000017a91407c95ac481802bb9129abc7c3b6c4ccf7fdb18808700e1f5050000000017a914ff536bd86b1701410a8074bfa5da114dc1bcef5e8702473044022079aa0013b2d2ca870c7d3b52f1f17b77fb53e8c11b36cdce1a9b2802c89d448802204d000486d475ea3faaef06f8dc640617fbad1300be2d56032733bddfd153a68e01210314cec2b73d7865554856c0c2e429be5566d8565e1f5ad9c6412b3022cbe65d1a67000000020000000001015aafbfa67849221ee4fa3e0309589913d182f90f9b3e8b8e3c4e066958defefb010000001716001432287953b7fa55d096fa2ed8a8e9b0104ec08d11feffffff02183f7fe20000000017a914b10a7f4ddb41b282e8802c4b740c70728927d1dc8700e1f5050000000017a914c4abadf18e28efb488fb987f40ab8396257758aa870247304402202438b022f4cb03f014a613f70c5a4ccc89f0805fcb45d3ba9e458b32fb9e98ec02203c9e47d3e0a398d054ed536635cc7aa844c2c6b2597455361b79d442f88bed27012102a26fd1a7b45f8c5ca99c927df7e5acaf5994c62f79aa5f1ab5d55ae6bf2f4d0d670000000200000000010115a7808c4aa756b61a946545402053b697b647281736e1db4b70f7f9683feeb40000000017160014d46e7c4b226f46575ac96377b3e0adb990f5c656feffffff0290d9e31f0100000017a914545263be9c28f847ef7f485cbb3d33b7e5c4c59f87801d2c040000000017a914d3281d206f3fab6e5c5f94b28f622966f3caedc98702473044022003d2b2c2e15d4e3df9bcccb5992b1feffdad7f67b734b24118da317f0dd11f9b022012933ac8972309b549a9e4ab4078b186981bd7623967707d6673ab650a7da97f0121025cb0e64a4a6f6a5552c97527f847c2befa52b3a97cbf77ec2e3c652c9e51f33967000000020000000001010ca1e9cb8e888125e60c0bb84632bcda54bcea9bb51419e5d053f110995362df0100000017160014d8d9152f2f59bb080c57234bdd1e8da28dff550efeffffff029896751e0100000017a9141cf6695ef6f905b11112372dcbe01c786f78f2e68740db1c040000000017a9147a5f842f5c55e838cd5fce3f56f9451e89eeb84787024730440220630ea5630d6e5029a8b8c23a7d8a43090beab49e0a1b28b7487464c0a150ab8e02207c1a257c6fcadaa44c0cfa871c9829a8adb7ccf9c7d5531695afa145bfb02439012103932bfea3878d3648b73c6c0beb32ccb5c2187f200c35ab78fbf3416678fccf356700000002000000000101269198c3ffaa45b5841c7b74daa431d4fa37f73196d29b336f792893b357f37b0000000017160014260a2496f7c8b5022413b6e093ccf775e27120ccfeffffff02a04184df0000000017a9147d2e00d1257ca06d92776a59db863f322bc2348b8780f0fa020000000017a914629149372520a6628d6e283b930f3d451a7b41748702473044022076d92f685bf1075356cbc535d9816e8a5488b25adccf9f7889560dc9dd18f3a3022046d5b858a8a6b7a3551b1b64bd35d01b32a331e892f4fee9345c7544e772a633012103347b2450980be5de51522e26d38f7ed3c7da97a82f083bfe6fc1ce6cc697b99867000000020000000001017e8b8d8afbd2ffbc0fce8f1531ba95ccd98ed81898a8376c86839ec92dc58b2a00000000171600146d8357cfd20cd6deabfab19c5704220881beafa5feffffff0240787d010000000017a914904dabdd96203bcab0f5ab833052c33a24db7baf8768bc06de0000000017a914e3d5a402751bd735422ab1e9990d12355dbb3405870247304402205034b2d8e91c4ca64ec5f160185bc558adb01c1306ad511bfe996e21c4bbc2bc02205b65790d64854feeb57083c8289e6499edfde17e68ed3e8921f03fd3eb86f91601210253a348408365169532b7962e509d9ae7d4ac61d1aa9339ec4b2f594e335a278e67000000'
block_list = break_the_block(block)
block_list

['00000020f67472a866d2b526a22ab132c2ad46578d58c968a3ff4542bc31b89778b2f237032f0b5b9a90f0bf15bf4396f9d8733b55a8cef1e8dbcdbced04e2a3ddae2b33cd11295effff7f200100000009020000000001010000000000000000000000000000000000000000000000000000000000000000ffffffff0401680101ffffffff02c059062a0100000017a9148b7a1532abef9a6cfc79cbb89a0fb171c73cb5d7870000000000000000266a24aa21a9edcc9e9dd007027c2ee8606ab265410f0af99d08038ac1a2298577f1fd47769c7901200000000000000000000000000000000000000000000000000000000000000000000000',
 '00020000000001010de9e45636dc1b3178cdf43311bb3dc31d88e14e400450648a663f2dc9e64088000000001716001446d96422bc6661867305deb392a8c5c07d18f4c4feffffff02080410240100000017a914fb5ca92fef360be3009d9a8348212d35c91bf65c8700e1f5050000000017a914502f9eb830f1ac210a79147721b8c04f881dbe71870247304402204c8c27a2d230bb5b87c4732c4f0c3a63cfe61a5e3cdcc27e4ba3fa58400b314e02206e72f5670fa0d698ef1306b791794ef4a5188dd6b0555b6a10dc25b759fbdf7901210314cec2b73d7865554856c0c2e429be5566d8565e1f5ad9c6412b3022cbe65d1a67000

In [4]:
#This cell creates a random set of 20 secret values for S, values will range between 100 and 1000
def create_secret_value_table():
    seed(42)
    S_list = []
    for i in range(20):
        S_list.append(randint(100,1000))
    S_list.sort()
    return S_list
    

In [5]:
#This cell creates a table of 20 key pairs to be combined with the secret list to act as a reference. NOTE SCRIPT REQUIRES 4 CORE CPU
def generate_RSA_keypairs():
    cols = ['public key', 'private key']
    N = 20
    RSA_list = pd.DataFrame(columns = cols)
    for i in range(N):
        (pubkey, privkey) = rsa.newkeys(4096, poolsize = 6)
        RSA_list = RSA_list.append({'public key' : pubkey , 'private key' : privkey}, ignore_index = True)
    return RSA_list

In [ ]:
#This function creates a master key for the secrets table. Deciding against second round of encryption for this piece
#def generate_encrypted_keys():
#    (master_pub, master_priv) = rsa.newkeys(4096, poolsize 6)
#    S_RSA_slice = S_RSA_DF.copy()
#    S_RSA_slice.columns= ['SecretID', 'PubKey', 'PrivKey']
#    S_RSA_slice['EncryptedPrivate'] = S_RSA_slice.apply(lambda row: rsa.encrypt(row.PrivKey, master_pub), axis = 1)
#    S_RSA_slice
 
    

In [73]:
#S_df = pd.DataFrame({"Secret ID": S_list})
#S_RSA_DF = pd.concat([S_df, RSA_list], axis = 1)

In [6]:
#Function to select the agents that will be signing
def we_need_agents():
    agent_list = []
    for i in range(1,11):
        agent_list.append(i)
    agent_list = random.sample(agent_list, 5)
    return agent_list
    

In [11]:
#Function to load the secrets csv if it has already been created
def load_secrets_blocks():
    S_RSA_DF = pd.read_csv('RSA_Keys.csv')
    Block_DF = pd.read_csv('Blockdata.csv')
    return S_RSA_DF, Block_DF

In [13]:
(S_RSA_DF, Block_DF) = load_secrets_blocks()

In [15]:
Block_DF.iloc[0]['Raw_bytes']

'00000020f67472a866d2b526a22ab132c2ad46578d58c968a3ff4542bc31b89778b2f237032f0b5b9a90f0bf15bf4396f9d8733b55a8cef1e8dbcdbced04e2a3ddae2b33cd11295effff7f200100000009020000000001010000000000000000000000000000000000000000000000000000000000000000ffffffff0401680101ffffffff02c059062a0100000017a9148b7a1532abef9a6cfc79cbb89a0fb171c73cb5d7870000000000000000266a24aa21a9edcc9e9dd007027c2ee8606ab265410f0af99d08038ac1a2298577f1fd47769c790120000000000000000000000000000000000000000000000000000000000000000000000000020000000001010de9e45636dc1b3178cdf43311bb3dc31d88e14e400450648a663f2dc9e64088000000001716001446d96422bc6661867305deb392a8c5c07d18f4c4feffffff02080410240100000017a914fb5ca92fef360be3009d9a8348212d35c91bf65c8700e1f5050000000017a914502f9eb830f1ac210a79147721b8c04f881dbe71870247304402204c8c27a2d230bb5b87c4732c4f0c3a63cfe61a5e3cdcc27e4ba3fa58400b314e02206e72f5670fa0d698ef1306b791794ef4a5188dd6b0555b6a10dc25b759fbdf7901210314cec2b73d7865554856c0c2e429be5566d8565e1f5ad9c6412b3022cbe65d1a67000000020

In [8]:
#This cell generates two values fC1 and fC2 using random int for the generation of the 2nd degree polynomial 
def generate_function_coefficients():
    row = randint(0,19)
    S = S_RSA_DF.iloc[row]['Secret ID']
    fC1= randint(50,500)
    fC2= randint(50,500)
    return S, fC1, fC2

In [33]:
Block_DF

,Block,Block_Number,Block_hash,Raw_bytes
0,1,104,126ec36152ebc96bd909a76a27e2722b9f0225de4504d3...,00000020f67472a866d2b526a22ab132c2ad46578d58c9...
1,2,105,23206e58d35e3783bfb749c46feddf047a02a4eba9a781...,000000209176eb369dd06f601fd30445de25029f2b72e2...
2,3,106,32d31172520e4da4f382471d0693189662305d79aca279...,000000206982d12e2bf5a04a7b81a7a9eba4027a04dfed...
3,4,107,365c1d7d2daf781f4a45a478c85c13eabc757849402b18...,0000002033817683c52f3e1e3979a2ac795d3062961893...
4,5,108,08f28662ba1df3f21e19709c0e1c7d3691fd9cf4c7e780...,000000206968668ba924cb5577182b40497875bcea135c...
5,6,109,73a630a38045cdbe95b10f66183effc581b0458b74d0e5...,000000202a202fb6639691637b80e7c7f49cfd91367d1c...
6,7,110,7aade5e1b42196ac7226f1cc534887e664dfa131680a7a...,0000002072683a587a183840e8e5d0748b45b081c5ff3e...
7,8,111,10ab6e2f2a32f0c6e0c56fa381adbd7e409b1654c12ab7...,00000020bdda082fc3fee2e7507a0a6831a1df64e68748...
8,9,112,6dbf9d21a3a2db9549c5f809487d6ab9f508503fea1227...,00000020c0865edeb1123be76ab72ac154169b407ebdad...
9,10,113,6dfb64a83efaab0f3d623a5738a1fbd11d64cdf8f02b72...,0000002066e5b0143eadef5b362712ea3f5008f5b96a7d...


In [9]:
#This cell generates the polynomial function for evaluation of shares, Shamir_Poly is a polynomial variable
#def generate_polynomial_function:
#    return Shamir_Poly

SyntaxError: invalid syntax (<ipython-input-9-428ae4495048>, line 2)

In [10]:
#This cell calculates the shares off the Shamir_Poly. This function should take in the five agents that signed the transaction, calculate the shares for all five, then return their relevant value.
def generate_Shamir_Shares (agent_list, S, fC1, fC2):
    shamir_shares = []
    for agent in agent_list:
        share = S + fC1*agent + fC2 * agent * agent
        shamir_shares.append({agent:share})
    return shamir_shares

In [128]:
index = S_RSA_DF.loc[S_RSA_DF['Secret ID'] == S].index[0]
string1 = S_RSA_DF.loc[S_RSA_DF['Secret ID'] == S]['private key'][index]
print(string1)

PrivateKey(59097076057511595382855397596573846106480327664542480084619224661019374318200613241196049679453660782721992319993478195130544310725705063919601186127631895520665933066472595209692914301289657501730191518100806554468658168971825589145491301467717680256605648565083091544360125786365009045289536364704848510093693700923589259122072182847941987929593980880071371695826751435815792853100241957946138173747228650727859010027801376198390814915809201260308165319043424958518552372624598132319052901559386276480433500655331305061239863274915164981752478879318773164392719656367086180941722303605317834857410309779411858335740210598596595404552828127026598543799143226677946044900253243728921619230487201066399093705509570614726275355543701375691319492639632179457321134240591763610137475955658376858456860546073595687225660187321531368505528826112728304326632757622631900666178153930045207692489621666308595000801578865699655145439043350679159135491289337302118708854177275780180796651550616056432

In [29]:
def encrypt_the_block(block, S):
    message = block.encode('utf-8')
    index = S_RSA_DF.loc[S_RSA_DF['Secret ID'] == S].index[0]
    string1 = S_RSA_DF.loc[S_RSA_DF['Secret ID'] == S]['private key'][index]
    list1 = string1.split(', ')
    list1[0] = list1[0][11:]
    list1[-1] = list1[-1][:-1]
    n = int(list1[0])
    e = int(list1[1])
    d = int(list1[2])
    p = int(list1[3])
    q = int(list1[4])
    priv_key = rsa.PrivateKey(n, e, d, p, q)
    encrypted_block = rsa.encrypt(message, priv_key)
    return encrypted_block

In [63]:
share_list

[{9: 23959}, {1: 1239}, {7: 15435}, {8: 19460}, {4: 6204}]

In [127]:
S = 792
index = S_RSA_DF.loc[S_RSA_DF['Secret ID'] == S].index[0]
string1 = S_RSA_DF.loc[S_RSA_DF['Secret ID'] == S]['private key'][index]
list1 = string1.split(', ')
list1[0] = list1[0][11:]
list1[-1] = list1[-1][:-1]
print(list1)

['59097076057511595382855397596573846106480327664542480084619224661019374318200613241196049679453660782721992319993478195130544310725705063919601186127631895520665933066472595209692914301289657501730191518100806554468658168971825589145491301467717680256605648565083091544360125786365009045289536364704848510093693700923589259122072182847941987929593980880071371695826751435815792853100241957946138173747228650727859010027801376198390814915809201260308165319043424958518552372624598132319052901559386276480433500655331305061239863274915164981752478879318773164392719656367086180941722303605317834857410309779411858335740210598596595404552828127026598543799143226677946044900253243728921619230487201066399093705509570614726275355543701375691319492639632179457321134240591763610137475955658376858456860546073595687225660187321531368505528826112728304326632757622631900666178153930045207692489621666308595000801578865699655145439043350679159135491289337302118708854177275780180796651550616056432653318831